In [2]:
import pandas as pd
import openpyxl as pyx


# wb = pyx.load_workbook('../sample_data/FHN_pkg_analyzer_example.xlsx')

# print(wb.sheetnames)






In [2]:
# sheet = wb.active

# [cell.value.replace('\n','').strip() for cell in sheet[13] if cell.value != None]

In [8]:
test = pd.read_csv('../sample_data/SAMPLE_FHN.csv').head()

In [ ]:
cols = test.columns.to_list()
cols = [c.strip().replace('\n'," ") for c in cols]
cols = [c for c in test.columns.to_list() if "unnamed" not in str(c).lower()]
test = test[[c for c in test.columns.to_list() if "unnamed" not in str(c).lower()]]

test.columns = cols
test

In [91]:
import pandas as pd
import numpy as np
import openpyxl as pyxl
from dataclasses import dataclass
import json
from pathlib import Path
from difflib import SequenceMatcher

class LoanTape:

    df: pd.DataFrame
    raw_dfs: list
    naics: dict
    format_packages: dict

    def rm_unnamed(self, _cols:list)->list:
        return [c for c in _cols if "unnamed" not in str(c).lower()]

    def norm_raw_cols(self):
        """Normalize the columns -- remove white space and line breaks"""
        if len(self.raw_dfs)>0:
            norm_dfs = []
            for df in self.raw_dfs:
                cols = df.columns.to_list()
                cols = [c.strip().replace("\n"," ") for c in cols]
                df.columns = cols
                norm_dfs.append(df)
            self.raw_dfs = norm_dfs
        else:
            print("No raw loan data")      
        return None

    def load_format_packges(self):
        """parse the raw loan data by using the available packages"""
        pkg_path = Path('package_maps/packages.json')
        pkg_file = open(pkg_path)
        format_opts = json.load(pkg_file)
        return format_opts
    
    def __init__(self, clean_columns, data=list()):
        self.df = pd.DataFrame(columns=clean_columns)
        self.raw_dfs = data
        self.norm_raw_cols()
        naics_tbl = pd.read_csv('package_maps/NAICS_2017.csv')
        self.naics = dict(naics_tbl.values)
        self.raw_dfs = [df[self.rm_unnamed(df.columns.to_list())] for df in self.raw_dfs]
        self.format_packages = self.load_format_packges()

    def format_columns(self):
        """Use the format packages to reformat the raw data"""
        # get all available formats
        format_keys = self.format_packages.keys()
        for idx, df in enumerate(self.raw_dfs):
            # for each dataframe, get the columns
            temp_cols = set(df.columns.to_list())
            # for each possible format type:
            for key in format_keys:
                # get the unformatted column names from the format option
                format_type_keys = set(self.format_packages[key].keys())
                # see if the format matches ~90%
                match_ratio = 1-len(temp_cols - format_type_keys) / len(format_type_keys)
                if match_ratio < .9:
                    continue
                else:
                    # If match, format the dataframe
                    self.raw_dfs[idx] = df.rename(columns=self.format_packages[key])
                    self.raw_dfs[idx]['Industry'] = self.raw_dfs[idx]['SIC / NAICS'].map(self.naics)
                    break

    def combine_raw_dfs(self):
        temp = []
        for df in self.raw_dfs:
            existing_cols= [c for c in df.columns if c != ""]
            temp.append(df[existing_cols])
        self.df = pd.concat(temp, ignore_index=True)
        self.df = self.df[self.df['GP#'].notna()]


        
        

In [92]:
import json
from pathlib import Path

clean_columns= ['Pck / Deal','GP#', 'Days', 'Category', 'Borrower Name', 'City', 'State', 'SIC /NAICS', 'ADJ', 'Accrual', 'Note Date',
'Note Maturity', 'Int. Paid to Date', 'Loan Spread', 'Loan Rate',
'Strip Rate', 'Original Balance', 'Current Balance', 'Multiple', 
'Proceeds', 'Term', 'Age', 'Rmos', 'Industry', 'Prepayment Penalty',
'Term Bucket', 'Industry Bucket', 'Lender', 'Prepayment Notice']

pkg_path = Path('package_maps/packages.json')
pkg_file = open(pkg_path)
pkgs = json.load(pkg_file)

test_FHN = pd.read_csv('../sample_data/SAMPLE_FHN.csv').head()
test_FHN_2 = pd.read_csv('../sample_data/SAMPLE_FHN.csv').tail()

lt = LoanTape(clean_columns=clean_columns,data=[test_FHN, test_FHN_2])

lt.format_columns()

lt.combine_raw_dfs()

# test = lt.raw_dfs.pop()
# fmt = lt.parse_data()

# for key in fmt.keys():
#     fmt_keys=(set(fmt[key].keys()))
#     samp= set(test.columns.to_list())
#     match_ratio = 1-len(samp-fmt_keys) / len(fmt_keys)
#     if match_ratio > .9:
#         print('match {key}'.format(key=key) )
#         test = test.rename(columns=fmt[key])

# # for c in test.columns:
# naics = pd.read_csv('package_maps/NAICS_2017.csv')
# naics= (dict(naics.values))
# test['Industry'] = test['SIC / NAICS'].apply(lambda x: naics[x])



,GP#,Current Balance,Strip Rate,Note Date,Note Maturity,Multiple,Borrower Name,func_geosplit,SIC / NAICS,Gross Cpn,Loan Spread,ADJ,Accrual,Lender,Industry
0,6.050028e+09,3258532.15,3.405,44064.0,53523.0,3.3,"Rainflow Investments, Inc.","Manteca, CA 95337",811192.0,2.75,-2.350,Q,365/365,NaN,Car Washes
1,4.019269e+09,1902000.00,0.305,44707.0,54132.0,3.3,BBC Investments/Incendiary Brewing Company,"Lewisville, NC 27023",312120.0,1.50,-0.500,Q,365/365,NaN,Breweries
2,2.976569e+09,858627.60,0.055,44587.0,53950.0,3.3,"Hanning & Sacchetto Properties, LLC","Whittier, CA 90602",541110.0,1.25,-0.500,Q,365/365,NaN,Offices of Lawyers
3,3.718729e+09,318670.42,1.790,44652.0,54011.0,3.3,121 Healthcare Services LLC,"Fort Lauderdale, FL 33309",621111.0,2.75,-0.675,Q,30/360,NaN,Offices of Physicians (except Mental Health Sp...
4,4.128449e+09,253553.36,1.355,44742.0,54011.0,3.3,Bronson Heritage Chiropractic LLC,"Azle, TX 76020",621310.0,2.00,-0.500,Q,365/365,NaN,Offices of Chiropractors
5,1.572677e+09,3044945.60,1.655,43118.0,52277.0,3.3,Ayra Hotels II LLC DBA Sleep Inn,"SE Marietta, GA 30067.",721110.0,1.00,-0.675,Q,30/360,NaN,Hotels (except Casino Hotels) and Motels
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
test_FHN_2 = pd.read_csv('../sample_data/SAMPLE_FHN.csv').tail()
test_FHN_2

,Trans,GP#,Status,Inv Status,Pkg ID,GTD Bal,Orig Fee,Note DT,Loan Age,Matures,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
232,320160.11,1.572677e+09,Tentative,Available,969.0,3044945.6,1.655,43118.0,60.0,52277.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

# Dictionary with column names as keys and corresponding values
fmt = {'A': 'Column 1', 'B': 'Column 2', 'D':'Not Here'}

# Set column names using rename() function
df = df.rename(columns=fmt)

# Print the updated DataFrame
print(df)

   Column 1  Column 2  C
0         1         4  7
1         2         5  8
2         3         6  9


In [ ]:
def geo_split(**kwargs):
    print(kwargs.keys())


geo_split(format='FHN')

dict_keys(['format'])
